In [14]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.utils.data as data_utils
import numpy as np
import pandas as pd
import math

In [4]:
input_size=4860
sequence_length=28
num_layers=3
hidden_size=1024

learning_rate = 0.001
num_epochs = 5

num_classes =10
batch_size = 64

In [5]:
class SimpleGRU(nn.Module):
    def __init__(self, input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes, sequence_length=sequence_length):
        super(SimpleGRU, self).__init__()
        self.hidden_size  = hidden_size
        self.num_layers = num_layers
        
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size * sequence_length, num_classes)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out,_ = self.gru(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc1(out)
        return out

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleGRU().to(device=device)

# Data Prep

In [61]:
df = pd.DataFrame()
x = pd.read_csv('./GRU_data/chembl_klek.csv', sep=',')
df['x'] = x.fps.apply(eval)
df['y'] = pd.read_csv('./GRU_data/selfies.csv', sep=' ')

In [77]:
def create_fp_array(fp, length):
    fp_array = np.zeros(length)
    for i in fp:
        fp_array[i] = 1
    return fp_array

loaded_fps = []
for item in df['x']:
    print(item)
    fp = create_fp_array(item, 4860)
    loaded_fps.append[fp]
df['x'] = loaded_fps

[668, 676, 838, 1012, 1023, 1145, 1192, 1317, 1451, 1460, 1641, 1723, 1749, 2780, 2854, 2885, 2974, 3012, 3024, 3528, 3533, 3881, 3942, 4079, 4121, 4236, 4245, 4246, 4286, 4319, 4325, 4327, 4328, 4330, 4546, 4590, 4842, 4852]


TypeError: 'builtin_function_or_method' object is not subscriptable

In [53]:
print(df.shape())

dataset = torch.utils.data.TensorDataset(df['x'], df['y'])

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [0.9, 0.1])

import matplotlib.pyplot as plt
plt.scatter(train_dataset, marker='.')
plt.scatter(test_dataset, marker='.')

TypeError: 'tuple' object is not callable

In [31]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

NameError: name 'train_dataset' is not defined

In [ ]:
train_dataset

In [ ]:
loss_criterion  = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
current_loss = 0
for epoch in range(num_epochs):
    for data, target in train_dataloader:
        data = data.to(device=device)
        target = target.to(device=device)
        print(data.size())
        score = model(data)
        loss = loss_criterion(score, target)
        current_loss = loss
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
    print(f"At epoch: {epoch}, loss: {current_loss}")